In [13]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

In [38]:
openai_api_key = '...'

### Load up your HTML from your company page

In [28]:
def get_company_page(company_path):
    y_combinator_url = f"https://www.ycombinator.com{company_path}"
    
    print (y_combinator_url)

    loader = UnstructuredURLLoader(urls=[y_combinator_url])
    return loader.load()


# Get the data of the company you're interested in
data = get_company_page("/companies/gitlab")
    
print (f"You have {len(data)} document(s)")

https://www.ycombinator.com/companies/gitlab
You have 1 document(s)


In [6]:
print (f"Preview of your data:\n\n{data[0].page_content[:30]}")

Preview of your data:

Accelerator

About

Apply

FAQ


In [29]:
# Split up the texts so you don't run into token limits
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 800,
    chunk_overlap  = 0
)

In [30]:
docs = text_splitter.split_documents(data)

print (f"You now have {len(docs)} documents")

You now have 7 documents


### Write your custom prompt templates
These will be used for your specific use case:

1. `map_prompt` will be the prompt that is done on your first pass of your documents
2. `combine_prompt` will be the prompt that is used when you combine the outputs of your map pass

In [9]:
map_prompt = """Below is a section of a website about {prospect}

Write a concise summary about {prospect}. If the information is not about {prospect}, exclude it from your summary.

{text}

CONCISE SUMMARY:"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text", "prospect"])

In [10]:
combine_prompt = """
Your goal is to write a personalized outbound email from {sales_rep}, a sales rep at {company} to {prospect}.

A good email is personalized and combines information about the two companies on how they can help each other.
Be sure to use value selling: A sales methodology that focuses on how your product or service will provide value to the customer instead of focusing on price or solution.

INFORMATION ABOUT {company}:
{company_information}

INFORMATION ABOUT {prospect}:
{text}

INCLUDE THE FOLLOWING PIECES IN YOUR RESPONSE:
- Start the email with the sentence: "We love that {prospect} helps teams..." then insert what they help teams do.
- The sentence: "We can help you do XYZ by ABC" Replace XYZ with what {prospect} does and ABC with what {company} does 
- A 1-2 setenece description about {company}, be brief
- End your email with a call-to-action such as asking them to set up time to talk more

YOUR RESPONSE:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["sales_rep", "company", "prospect", \
                                                                         "text", "company_information"])

In [11]:
company_information = """
* RapidRoad helps product teams build product faster
* We have a platform that allows product teams to talk more, exchange ideas, and listen to more customers
* Automated project tracking: RapidRoad could use machine learning algorithms to automatically track project progress, identify potential bottlenecks, and suggest ways to optimize workflows. This could help product teams stay on track and deliver faster results.
* Collaboration tools: RapidRoad could offer built-in collaboration tools, such as shared task lists, real-time messaging, and team calendars. This would make it easier for teams to communicate and work together, even if they are in different locations or time zones.
* Agile methodology support: RapidRoad could be specifically designed to support agile development methodologies, such as Scrum or Kanban. This could include features like sprint planning, backlog management, and burndown charts, which would help teams stay organized and focused on their goals.
"""

### LangChain Magic

In [36]:
llm = OpenAI(temperature=.7, openai_api_key=openai_api_key)

chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=map_prompt_template,
                             combine_prompt=combine_prompt_template,
#                              verbose=True
                            )

In [37]:
output = chain({"input_documents": docs, # The seven docs that were created before
                "company": "RapidRoad", \
                "company_information" : company_information,
                "sales_rep" : "Greg", \
                "prospect" : "GitLab"
               })

In [35]:
print (output['output_text'])


Dear GitLab,

We love that GitLab helps teams unlock from the constraints of current toolchains, compress the time needed to plan and monitor changes, and deliver quality software quickly. At RapidRoad, we understand the importance of collaboration and communication in product development. We can help you deliver faster results, stay organized and focused on your goals, and facilitate communication among teams with our platform.

RapidRoad is a platform that allows product teams to talk more, exchange ideas, and listen to more customers. We offer automated project tracking, collaboration tools, and agile methodology support so that you can build product faster.

We believe that together, RapidRoad and GitLab could be a powerful combination. Let's set up a time to chat and discuss how our teams can work together to bring even better results for your organization.

Sincerely,
Greg
RapidRoad Sales Rep


### Rinse and Repeat: Loop Through Companies

In [18]:
import pandas as pd

In [19]:
df_companies = pd.read_clipboard()

In [20]:
df_companies

,Name,Link
0,Findly,/companies/findly
1,Maya Labs,/companies/maya-labs
2,Phind,/companies/phind
3,Juicebox,/companies/juicebox
4,Penguin AI,/companies/penguin-ai
5,Poly,/companies/poly
6,Olli.ai,/companies/olli-ai


In [21]:
for i, company in df_companies.iterrows():
    print (f"{i + 1}. {company['Name']}")
    page_data = get_company_page(company['Link'])
    docs = text_splitter.split_documents(page_data)

    output = chain({"input_documents": docs, \
                "company":"RapidRoad", \
                "sales_rep" : "Greg", \
                "prospect" : company['Name'],
                "company_information" : company_information
               })
    
    print (output['output_text'])
    print ("\n\n")

1. Findly
https://www.ycombinator.com/companies/findly

Dear Findly Team,

We love that Findly helps teams quickly and accurately access data insights without having to learn SQL or Python. We can help you provide more actionable insights by using our platform to talk more, exchange ideas, and listen to more customers. 

RapidRoad is a platform that helps product teams build product faster. It offers automated project tracking, collaboration tools, and agile methodology support to help teams stay organized and deliver faster results. 

We believe that our platform and your AI chatbot will be a great combination to help teams develop products faster and with actionable insights. Let's talk soon and see how we can work together to make this happen. 

Best, 
Greg 
Sales Rep, RapidRoad



2. Maya Labs
https://www.ycombinator.com/companies/maya-labs

Dear Maya Labs, 

We love that Maya Labs helps teams build custom software in minutes using natural language. We at RapidRoad are dedicated to